In [ ]:
# Import required libraries and dependencies
import pandas as pd
import hvplot.pandas
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
import warnings
warnings.simplefilter("ignore", category=FutureWarning)

In [ ]:
# Load the data into a Pandas DataFrame
df_market_data = pd.read_csv(
    "Resources/crypto_market_data.csv",
    index_col="coin_id")

# Display sample data
df_market_data.head(10)

In [ ]:
# Generate summary statistics
df_market_data.describe()

In [ ]:
# Plot your data to see what's in your DataFrame
df_market_data.hvplot.line(
    width=800,
    height=400,
    rot=90
)

---

### Prepare the Data

In [ ]:
# Use the `StandardScaler()` module from scikit-learn to normalize the data from the CSV file
market_data_scaled = StandardScaler().fit_transform(df_market_data[["price_change_percentage_24h", "price_change_percentage_7d",
                                                                    "price_change_percentage_14d", "price_change_percentage_30d",
                                                                    "price_change_percentage_60d", "price_change_percentage_200d", "price_change_percentage_1y"]])

In [ ]:
# Create a DataFrame with the scaled data
df_market_data_scaled = pd.DataFrame(market_data_scaled, columns=["price_change_percentage_24h", "price_change_percentage_7d",
                                                                    "price_change_percentage_14d", "price_change_percentage_30d",
                                                                    "price_change_percentage_60d", "price_change_percentage_200d", "price_change_percentage_1y"])

# Copy the crypto names from the original data
df_market_data_scaled["coin_id"] = df_market_data.index

# Set the coinid column as index
df_market_data_scaled = df_market_data_scaled.set_index("coin_id")

# Display sample data
df_market_data_scaled.head()

---

### Find the Best Value for k Using the Original Data.

In [ ]:
# Create a list with the number of k-values from 1 to 11
k = list(range(1, 11))

In [ ]:
# Create an empty list to store the inertia values
inertia = []

# Create a for loop to compute the inertia with each possible value of k
# Inside the loop:
# 1. Create a KMeans model using the loop counter for the n_clusters
# 2. Fit the model to the data using `df_market_data_scaled`
# 3. Append the model.inertia_ to the inertia list
for i in k:
    model = KMeans(n_clusters=i, random_state=0)
    model.fit(df_market_data_scaled)
    inertia.append(model.inertia_)

In [ ]:
# Create a dictionary with the data to plot the Elbow curve
elbow_market_data = {
    "k": k,
    "inertia": inertia
}

# Create a DataFrame with the data to plot the Elbow curve
df_elbow_market = pd.DataFrame(elbow_market_data)

In [ ]:
# Plot a line chart with all the inertia values computed with 
# the different values of k to visually identify the optimal value for k
elbow_plot_market = df_elbow_market.hvplot.line(x="k", y="inertia", title="Elbow Curve Using Market Data", xticks=k)
elbow_plot_market

#### Answer the following question: 

**Question:** What is the best value for `k`?

**Answer:** Based on this Elbow Curve, `k=4` is the best value. 

---

### Cluster Cryptocurrencies with K-means Using the Original Data

In [ ]:
# Initialize the K-Means model using the best value for k
model = KMeans(n_clusters=4)

In [ ]:
# Fit the K-Means model using the scaled data
model.fit(df_market_data_scaled)

In [ ]:
# Predict the clusters to group the cryptocurrencies using the scaled data
crypto_clusters = model.predict(df_market_data_scaled)

# Print the resulting array of cluster values
print(crypto_clusters)

In [ ]:
# Create a copy of the DataFrame
df_market_data_predictions = df_market_data_scaled.copy()

In [ ]:
# Add a new column to the DataFrame with the predicted clusters
df_market_data_predictions["CryptoCluster"] = crypto_clusters

# Display sample data
df_market_data_predictions.head()

In [ ]:
# Create a scatter plot using hvPlot by setting 
# `x="price_change_percentage_24h"` and `y="price_change_percentage_7d"` 
# Color the graph points with the labels found using K-Means and 
# add the crypto name in the `hover_cols` parameter to identify 
# the cryptocurrency represented by each data point.
df_market_data_predictions.hvplot.scatter(
    x="price_change_percentage_24h",
    y="price_change_percentage_7d",
    hover_cols = ["coin_id"],
    by="CryptoCluster",
    title = "Scatter Plot by  Market - k=4"
)

---

### Optimize Clusters with Principal Component Analysis.

In [ ]:
# Create a PCA model instance and set `n_components=3`
pca = PCA(n_components=3)

In [ ]:
# Use the PCA model with `fit_transform` to reduce to 
# three principal components.
markets_pca_data = pca.fit_transform(df_market_data_scaled)

# View the first five rows of the DataFrame 
markets_pca_data[:5]

In [ ]:
# Retrieve the explained variance to determine how much information 
# can be attributed to each principal component
pca.explained_variance_ratio_

In [ ]:
# Import dependency to use 'sum' function 
import numpy as np

In [ ]:
# Calculate total explained variance by adding the principal components 
total_explained_variance = np.sum(pca.explained_variance_ratio_)

# Display the Total Explained Variance
print("Total Explained Variance for the three principal components:", total_explained_variance)

#### Answer the following question: 

**Question:** What is the total explained variance of the three principal components?

**Answer:** Approximately 89.5% 

In [ ]:
# Create a new DataFrame with the PCA data
df_market_data_pca = pd.DataFrame(markets_pca_data, columns=["PC1", "PC2", "PC3"])

# Copy the crypto names from the original data
df_market_data_pca["coin_id"] = df_market_data.index

# Set the coinid column as index
df_market_data_pca = df_market_data_pca.set_index("coin_id")

# Display sample data
df_market_data_pca.head()

---

### Find the Best Value for k Using the PCA Data

In [ ]:
# Create a list with the number of k-values from 1 to 11
k = list(range(1, 11))

In [ ]:
# Create an empty list to store the inertia values
inertia = []

# Create a for loop to compute the inertia with each possible value of k
# Inside the loop:
# 1. Create a KMeans model using the loop counter for the n_clusters
# 2. Fit the model to the data using `df_market_data_pca`
# 3. Append the model.inertia_ to the inertia list
for i in k:
    model = KMeans(n_clusters=i, random_state=0)
    model.fit(df_market_data_pca)
    inertia.append(model.inertia_)

In [ ]:
# Create a dictionary with the data to plot the Elbow curve
elbow_data_pca = {
    "k": k,
    "inertia": inertia
}

# Create a DataFrame with the data to plot the Elbow curve
df_elbow_pca = pd.DataFrame(elbow_data_pca)

In [ ]:
# Plot a line chart with all the inertia values computed with 
# the different values of k to visually identify the optimal value for k.
elbow_plot_pca = df_elbow_pca.hvplot.line(x="k", y="inertia", title="Elbow Curve Using PCA Data", xticks=k)
elbow_plot_pca

#### Answer the following questions: 

* **Question:** What is the best value for `k` when using the PCA data?

  * **Answer:** 4


* **Question:** Does it differ from the best k value found using the original data?

  * **Answer:** No

### Cluster Cryptocurrencies with K-means Using the PCA Data

In [ ]:
# Initialize the K-Means model using the best value for k
model = KMeans(n_clusters=4)

In [ ]:
# Fit the K-Means model using the PCA data
model.fit(df_market_data_pca)

In [ ]:
# Predict the clusters to group the cryptocurrencies using the PCA data
crypto_clusters = model.predict(df_market_data_pca)

# Print the resulting array of cluster values.
print(crypto_clusters)

In [ ]:
# Create a copy of the DataFrame with the PCA data
df_markets_pca_predictions = df_market_data_pca.copy()

# Add a new column to the DataFrame with the predicted clusters
df_markets_pca_predictions["CryptoCluster"] = crypto_clusters

# Display sample data
df_markets_pca_predictions.head()

In [ ]:
# Create a scatter plot using hvPlot by setting 
# `x="PC1"` and `y="PC2"`. 
# Color the graph points with the labels found using K-Means and 
# add the crypto name in the `hover_cols` parameter to identify 
# the cryptocurrency represented by each data point
df_markets_pca_predictions.hvplot.scatter(
    x="PC1",
    y="PC2",
    by="CryptoCluster",
    hover_cols = ["coin_id"],
    title = "Scatter Plot by Market - PCA=3"
)

### Visualize and Compare the Results

In this section, you will visually analyze the cluster analysis results by contrasting the outcome with and without using the optimization techniques.

In [ ]:
# Composite plot to contrast the Elbow curves 
# Combine the plots using the + operator
elbow_composite_plot = elbow_plot_market + elbow_plot_pca

# Add title
elbow_composite_plot.opts(title="Comparison of Elbow Curves")

# Display the composite plot
elbow_composite_plot

In [ ]:
# Composite plot to contrast the clusters
scatter_market_data = df_market_data_predictions.hvplot.scatter(
    x="price_change_percentage_24h",
    y="price_change_percentage_7d",
    hover_cols=["coin_id"],
    by="CryptoCluster",
    title="Scatter Plot by Market - k=4"
)

scatter_pca_data = df_markets_pca_predictions.hvplot.scatter(
    x="PC1",
    y="PC2",
    by="CryptoCluster",
    hover_cols=["coin_id"],
    title="Scatter Plot by Market - PCA=3"
)

# Combine the plots using the + operator
clusters_composite_plot = scatter_market_data + scatter_pca_data

# Add title
clusters_composite_plot.opts(title="Comparison of Cluster Plots")

# Display the composite plot
clusters_composite_plot

#### Answer the following question: 

  * **Question:** After visually analyzing the cluster analysis results, what is the impact of using fewer features to cluster the data using K-Means?

  * **Answer:** For this specific analysis, the use of fewer features to cluster the data using K-Means results in tighter clusters than clustering the original data using K-Means. This indicates a potential reduction in the dimensionality of the data and less variability. This might be preferable for identifying distinct groups with less granular insight, which could either be a benefit or a hindrance depending on the task. 